In [47]:
import pandas as pd
import numpy as np
import urllib, json

In [48]:
df = pd.read_csv('../tabel/MAIN TABLE PREMIUM.csv', index_col=0)
df.columns

Index(['match_id', 'start_time', 'radiant_team_id', 'radiant_score',
       'dire_team_id', 'dire_score', 'radiant_win', 'radiant_H1', 'radiant_H2',
       'radiant_H3', 'radiant_H4', 'radiant_H5', 'dire_H1', 'dire_H2',
       'dire_H3', 'dire_H4', 'dire_H5', 'radiant_P1', 'radiant_P2',
       'radiant_P3', 'radiant_P4', 'radiant_P5', 'dire_P1', 'dire_P2',
       'dire_P3', 'dire_P4', 'dire_P5', 'league_name'],
      dtype='object')

In [120]:
# Создать главный дф и выбросить лишнее
main = df.loc[:, 'match_id':'radiant_win' ]
main.head(3)
# пробный 
prob = main[:1]

In [121]:
# создать дф из json по всем командам с OPENDOTA https://api.opendota.com/api/teams
url = "https://api.opendota.com/api/teams"
sauce = urllib.request.Request(url, headers={'User-agent': 'Mozilla/5.0'})
dat = json.loads(urllib.request.urlopen(sauce).read().decode())
# дв с именем команды и ее id
df_team_id = pd.DataFrame(dat).loc[:,['name', 'team_id']]

In [122]:
for index in prob.index:
    try:
        prob.loc[index, 'radiant_name'] = (
            df_team_id.loc[df_team_id['team_id'] == prob.loc[index, 'radiant_team_id'], 'name'].get_values()[0])
    except:
        print ('Остутсвует имя редиант')
        print('id команды - ', prob.loc[index, 'radiant_team_id'])
    try:
        prob.loc[index, 'dire_name'] = (
            df_team_id.loc[df_team_id['team_id'] == prob.loc[index, 'dire_team_id'], 'name'].get_values()[0])
    except:
        print ('Остутсвует имя даер')
        print('id команды - ', prob.loc[index, 'radiant_team_id'])
prob = prob.dropna()

C:\Sneg\Add.Program\Anaconda3\lib\site-packages\pandas\core\indexing.py:337: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Sneg\Add.Program\Anaconda3\lib\site-packages\pandas\core\indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [127]:
prob

,match_id,start_time,radiant_team_id,radiant_score,dire_team_id,dire_score,radiant_win,radiant_name,dire_name
0,2897064589,1483677730,1883502.0,23,350190.0,12,True,Virtus.pro,Fnatic


In [134]:
from datetime import date
# достать дату матча
date_match = prob['start_time'][0]

# работа с датами
a = date.fromtimestamp(date_match)
name_file = '{}-{}-{}.csv'.format(a.year, a.month, a.day-1, )
# загрузить файл с рейтингом на предыдущий день
raiting_team_date = pd.read_csv('../tabel with teams DATDOTA (elo, glicko)/{}'.format(name_file),  index_col=0)



In [144]:
index_radiant = raiting_team_date[raiting_team_date['team_Name'] == prob['radiant_name'][0]]
index_radiant.drop(['phi_glicko', 'sigma_glicko2'],  axis=1)
#raiting_team_date.iloc[index_radiant]

,team_Name,current_elo32,sevenDayAgo_elo32,sevenDayAvg_elo32,thirtyDayAgo_elo32,thirtyDayAvg_elo32,current_elo64,sevenDayAgo_elo64,sevenDayAvg_elo64,thirtyDayAgo_elo64,thirtyDayAvg_elo64,mu_glicko,rating_glicko,ratingSevenDaysAgo_glicko,sigma_glicko,mu_glicko2,phi_glicko2,rating_glicko2,ratingSevenDaysAgo_glicko2
1,Virtus.pro,1239.223638,1251.027919,1245.085169,1325.33498,1269.773656,1328.175555,1344.369092,1336.216614,1492.25145,1374.318822,2115.730552,1834.931156,1795.528506,112.319759,2002.113328,42.886073,1894.898146,1890.862803


In [43]:
# привязать team id к team_name
radiant_id = prob['radiant_team_id'][0]
radiant_id

1883502.0